In [1]:
!pip install librosa


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate

from sklearn.preprocessing import StandardScaler

In [3]:
folder_path = 'songs'
features = []

total_files = len([filename for filename in os.listdir(folder_path) if filename.endswith('.wav')])

with tqdm(total=total_files) as pbar:
    for filename in os.listdir(folder_path):
        if filename.endswith('.wav'):
            file_path = os.path.join(folder_path, filename)
            y, sr = librosa.load(file_path)

            # Mel-frequency Cepstral coefficients
            mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1)
            # Chroma features
            chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)
            # Spectral centroid
            spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
            # Spectral bandwith
            spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
            # Spectral contrast
            spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr))
            # Spectral rolloff
            spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
            # Tempo
            tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
            # Zero crossing rate
            zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y))

            features.append(np.hstack([mfccs, chroma, spectral_centroid, spectral_bandwidth, 
                                  spectral_contrast, spectral_rolloff, tempo, zero_crossing_rate]))
            
            pbar.update(1)
            
mfcc_headers = [f'mfcc_{i}' for i in range(13)]
chroma_headers = [f'chroma_{i}' for i in range(12)]

columns = []
columns.extend(mfcc_headers)
columns.extend(chroma_headers)
columns.extend(['spectral_centroid', 'spectral_bandwidth', 'spectral_contrast', 'spectral_rolloff',
              'tempo', 'zero_crossing_rate'])

feature_df = pd.DataFrame(features, columns=columns)

feature_df.to_csv('sound_features.csv', index=False)


100%|███████████████████████████████████████████| 10/10 [00:26<00:00,  2.70s/it]


In [4]:
sound_features_df = pd.read_csv('sound_features.csv')


In [5]:
display(sound_features_df)

,mfcc_0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,...,chroma_8,chroma_9,chroma_10,chroma_11,spectral_centroid,spectral_bandwidth,spectral_contrast,spectral_rolloff,tempo,zero_crossing_rate
0,-119.883514,78.195465,-5.079293,16.115835,8.802312,8.788719,5.017749,8.297569,2.037281,5.438093,...,0.426102,0.442033,0.362053,0.400779,2379.809563,2533.935542,22.005264,5261.087265,129.199219,0.091814
1,-165.491364,89.887207,9.622631,28.824188,9.862348,0.715233,7.598889,1.015978,5.371005,-0.741413,...,0.247188,0.379182,0.529390,0.309625,2328.919113,2609.601348,23.821849,5068.501473,99.384014,0.092295
2,-263.305817,86.074142,11.548472,23.479252,3.635492,5.045878,4.839182,0.582274,7.817739,0.376333,...,0.331482,0.491414,0.322831,0.371970,2561.512959,2673.636869,23.458048,5472.573941,117.453835,0.119242
3,-49.181473,69.625359,-11.549077,13.836099,3.922445,7.283914,5.275508,4.021071,3.152122,5.375659,...,0.389022,0.452060,0.409418,0.471878,2618.028685,2511.308062,20.885303,5376.408294,161.499023,0.126535
4,-135.907242,101.794563,25.623070,23.123831,8.992074,4.891822,9.471533,-0.671894,0.736529,10.573339,...,0.385096,0.272601,0.253039,0.363305,2107.592914,2431.577165,23.659678,4736.722383,103.359375,0.082603
5,-124.014420,103.057861,-1.579663,8.649143,-8.500657,-6.627197,-20.605783,-9.192368,-21.916517,-8.039808,...,0.300686,0.167178,0.274421,0.170304,1946.668092,2203.037515,26.128935,4069.927490,123.046875,0.081782
6,-137.621063,84.092110,-8.891517,28.446003,3.140309,9.002275,5.252365,12.670362,3.979600,8.366396,...,0.396250,0.481415,0.600927,0.426780,2362.140776,2440.904377,21.228489,4973.706413,89.102909,0.099254
7,-195.491211,130.852936,19.904680,22.960194,7.628928,0.067079,7.393571,-0.221847,-2.634048,1.128535,...,0.376539,0.340813,0.275992,0.403166,1446.082976,1916.769463,23.550778,2843.806541,92.285156,0.052020
8,-38.100956,70.222244,-2.880670,17.448526,1.117762,12.198992,-0.531046,6.038986,0.556414,6.931718,...,0.421182,0.360760,0.467224,0.392254,2729.146160,2631.424067,22.012557,5741.808757,107.666016,0.126865
9,-107.875107,83.763847,-4.584987,15.001758,-4.840187,-4.345690,-0.965797,-4.735468,-2.551290,-1.484672,...,0.215571,0.407293,0.398789,0.235425,2354.484402,2448.346537,24.313879,4916.218943,117.453835,0.101199


In [6]:
# Normalize sound features
scaler = StandardScaler()
X_sound = scaler.fit_transform(sound_features_df)

In [7]:
display(pd.DataFrame(X_sound))

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.221374,-0.666057,-0.689739,-0.585998,0.937192,0.877774,0.332603,1.087511,0.304205,0.500752,...,1.115767,0.655013,-0.248954,0.532259,0.271923,0.435850,-0.732490,0.521112,0.752904,-0.253400
1,-0.510053,0.007526,0.533089,1.441714,1.120276,-0.515430,0.645565,-0.127570,0.729885,-0.668856,...,-1.470396,-0.003071,1.273919,-0.517209,0.128329,0.787133,0.475840,0.279290,-0.728401,-0.231444
2,-2.078734,-0.212152,0.693270,0.588890,0.044804,0.231889,0.310952,-0.199943,1.042305,-0.457297,...,-0.251938,1.172055,-0.605893,0.200572,0.784623,1.084421,0.233852,0.786667,0.169359,0.999624
3,1.355245,-1.159797,-1.227862,-0.949747,0.094365,0.618097,0.363856,0.373891,0.446558,0.488935,...,0.579780,0.760002,0.182102,1.350834,0.944090,0.330801,-1.477450,0.665916,2.357651,1.332814
4,-0.035603,0.693531,1.863920,0.532180,0.969967,0.205304,0.872622,-0.409226,0.138114,1.472711,...,0.523037,-1.119018,-1.241050,0.100819,-0.496171,-0.039355,0.367970,-0.137311,-0.530894,-0.674199
5,0.155126,0.766312,-0.398659,-1.777365,-2.051293,-1.782479,-2.774236,-1.831041,-2.754426,-2.050238,...,-0.697097,-2.222852,-1.046459,-2.121251,-0.950242,-1.100363,2.010435,-0.974577,0.447238,-0.711700
6,-0.063088,-0.326340,-1.006820,1.381372,-0.040722,0.914626,0.361050,1.817201,0.552218,1.054998,...,0.684269,1.067360,1.924956,0.831614,0.222069,0.003947,-1.249174,0.160260,-1.239196,0.086477
7,-0.991170,2.367638,1.388295,0.506070,0.734531,-0.627279,0.620670,-0.334127,-0.292271,-0.314927,...,0.399337,-0.404806,-1.032159,0.559743,-2.362708,-2.429378,0.295533,-2.514166,-1.081093,-2.071366
8,1.532947,-1.125409,-0.506869,-0.373357,-0.390046,1.466269,-0.340185,0.710621,0.115115,0.783454,...,1.044646,-0.195954,0.708172,0.434114,1.257622,0.888446,-0.727639,1.124734,-0.316928,1.347867
9,0.413957,-0.345252,-0.648625,-0.763758,-1.419074,-1.388770,-0.392898,-1.087316,-0.281704,-0.809534,...,-1.927406,0.291270,0.085366,-1.371495,0.200465,0.038498,0.803122,0.088075,0.169359,0.175327


In [8]:
num_columns_X_sound = X_sound.shape[1]

In [9]:
num_genres = 3

In [10]:
# Dummy output values
y_genre = np.array([0, 1, 0, 1, 2, 2, 0, 0, 1, 0])
y_popularity = np.array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0])
y_danceability = np.array([0.23, 0.9, 0.4, 0.67, 0.5, 0.5, 0.3, 0.2, 0.12, 0.4])
y_energy = np.array([0.35, 0.56, 0.45, 0.75, 0.22, 0.9, 0.75, 0.3, 0.43, 0.5])

In [11]:
# Sound modality model
input_sound = Input(shape=(num_columns_X_sound,), name='input_sound')
sound_layer1 = Dense(64, activation='relu', name='sound_embedding')(input_sound)
sound_embedding = Dense(8, activation='relu', name='sound_output')(sound_layer1)

# TODO: Text modality model
# input_text = Input(shape=(num_columns_X_text,), name='input_text')
# text_layer1 = Dense(64, activation='relu', name='text_embedding')(input_text)
# text_embedding = Dense(8, activation='relu', name='text_output')(text_layer1)

# TODO: Image modality model
# input_image = Input(shape=(num_columns_X_image,), name='input_image')
# image_layer1 = Dense(64, activation='relu', name='image_embedding')(input_image)
# image_embeddng = Dense(8, activation='relu', name='image_output')(image_layer1)

# Combine the embeddings
# concatenated_output = Concatenate()([sound_embedding, text_embedding, image_embeddng])
concatenated_output = sound_embedding  # Only sound modality for now

# Four output layers
genre_output_layer = Dense(num_genres, activation='softmax', name='genre_output')(concatenated_output)
popularity_output_layer = Dense(1, activation='sigmoid', name='popularity_output')(concatenated_output)
danceability_output_layer = Dense(1, activation='sigmoid', name='danceability_output')(concatenated_output)
energy_output_layer = Dense(1, activation='sigmoid', name='energy_output')(concatenated_output)

In [12]:
# model = Model(inputs=[input_sound, input_text, input_image], outputs=[output_layer1, output_layer2, output_layer3, output_layer4])
model = Model(inputs=[input_sound], outputs=[genre_output_layer, popularity_output_layer, danceability_output_layer, energy_output_layer])
model.compile(optimizer='adam', loss={'genre_output': 'sparse_categorical_crossentropy', 'popularity_output': 'mean_squared_error', 'danceability_output': 'mean_squared_error', 'energy_output': 'mean_squared_error'}, 
              metrics={'genre_output': 'accuracy', 'popularity_output': 'mean_squared_error', 'danceability_output': 'mean_squared_error', 'energy_output': 'mean_squared_error'})

# Train model
# model.fit([X_sound, X_text, X_image], [y1, y2, y3, y4], epochs=10, batch_size=32, validation_split=0.2)
model.fit([X_sound], [y_genre, y_popularity, y_danceability, y_energy], epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1/1 [==============================] - 0s 429ms/step - loss: 1.5689 - genre_output_loss: 1.2684 - popularity_output_loss: 0.2191 - danceability_output_loss: 0.0505 - energy_output_loss: 0.0310 - genre_output_accuracy: 0.3750 - popularity_output_mean_squared_error: 0.2191 - danceability_output_mean_squared_error: 0.0505 - energy_output_mean_squared_error: 0.0310 - val_loss: 1.2188 - val_genre_output_loss: 0.8562 - val_popularity_output_loss: 0.2035 - val_danceability_output_loss: 0.0670 - val_energy_output_loss: 0.0921 - val_genre_output_accuracy: 0.5000 - val_popularity_output_mean_squared_error: 0.2035 - val_danceability_output_mean_squared_error: 0.0670 - val_energy_output_mean_squared_error: 0.0921
Epoch 2/10
1/1 [==============================] - 0s 11ms/step - loss: 1.5154 - genre_output_loss: 1.2196 - popularity_output_loss: 0.2151 - danceability_output_loss: 0.0504 - energy_output_loss: 0.0302 - genre_output_accuracy: 0.6250 - popularity_output_mean_squared_error: 0.2